In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Checking Coherence of Data Types in Data Frame

In [3]:
def col_types(df):
    for i in df.columns:
        print(
            df[i].map(type).value_counts(),
            '\n NaN values: ',
            df[i].isna().sum(),
            '\n'
    )

# 2. Possible Mappings Between Two Columns
Given 2 columns in a data frame, une might be interested in how they map onto each other. Is it a 1 to 1 mapping? Does it even define a function? And if so, is the function injective of surjective?

The following function, given col_1 and col_2, returns, for each value in col_1, how many values it is associated to in col_2:

In [4]:
# Here's an auxiliary function returning the list of indexes of an item in an iterable:
# (the funxtion says all positions at which an item can be found in a list)
def occurrences(seq, item):
    indexes = [index for (index, x) in enumerate(seq) if x == item]
    return indexes

In [5]:
# Try it out here:

# df = pd.DataFrame(
#     {
#         'a': [1, 2, 3, 4],
#         'b': [1, 1, 3, 5]
#     }
# )

In [6]:
def map_col2col(series_1, series_2):
    # given 2 Series of the same length, an element x of series_1 is said to map to y in series_2
    # if they happen have the same positional index at least once.
    # The functions returns a DataFrame whether each element of series_1
    # maps to each element of series_2.
    
    # Build a table where indexes are unique elements from series1 and columns unique elements from
    # series 2
    indexes = series_1.sort_values().unique()
    columns = series_2.sort_values().unique()
    matches = pd.DataFrame(index = indexes, columns = columns, data = False)
    
    for x in indexes:
        indexes_x_in_s1 = occurrences(series_1, x)   # positions of x in s1
        images_of_x_in_s2 = series_2[indexes_x_in_s1].unique()    # values y of s2 in the positions foud for x
        for y in columns:
            matches.loc[x, y] = y in images_of_x_in_s2
    return matches

In [7]:
# Try it out here:

# df1 = pd.DataFrame(
#     {
#         'a': [1, 2, 3, 4, 3],
#         'b': [1, 3, 1, 3, 1],
#         'c': ['lollo', 'gigio', 'lollo', 'gigio', 'lollo']
#     }
# )

# map_col2col(df1.b, df1.c)

By calling `map_count(col_1, col_2)` and then `map_count(col_2, col_1)`, one will be able to determine the relation between the two columns.

# 3. Rows of NaN
Are NaN values in col_1 also NaN in col 2?

In [8]:
def nan2nan(df, col_1_name, col_2_name):
    # Given 2 columns in a data frame df, this function
    # returns a list with the number of NaNs in the first,
    # The number of NaNs in the second,
    # And the number fo rows where both cols have NaN value
    
    col_1 = col_1_name 
    col_2 = col_2_name
    
    # Number of NaN rows in col_1:    
    nans_1 = pd.isna(df[col_1]).sum()
    # Number of NaN rows in col_2:
    nans_2 = pd.isna(df[col_2]).sum()
    # Number of rows where both col_1 and col_2 are NaN:
    nans_1and2 = (pd.isna(df[col_1]) * pd.isna(df[col_2])).sum()
    
    print(
    'NaN values in {}: '.format(col_1),
    nans_1,
    '\n\nNaN values in {}: '.format(col_2),
    nans_2,
    '\n\nNaN values in the same row for both {} and {}: '.format(col_1, col_2),
    nans_1and2
)

# 4. Foreign Keys Correspondance
Are you wandering whether two columns from different tables have the same unique values?

In [9]:
def foreign_k2k(series_1, series_2, df1_name = 'table1', df2_name = 'table2', n_matches = -1, mode = '>'):
    # Arguments: 
    #  1. two series passed as df1.series and df2.series.
    #  2. the name of df1 and the name of df2
    #  3. number of matches and a conditional selector
    # Returns by Default: a data frame of 2 columns, where the first is series_1. 
    # The second indicates for each element in series_1, the number of matches (same values)
    # found in series_2. If n_matches and mode are specified, only the values with the
    # selected number of matches are returned.
    # Example: n_matches = 0 and mode = '=' will return all values in series 1 that don't match
    # any value in series 2.
    #
    # mode possible values: '=', '<', '>', '!='
    
    # Defining column names for the output dataframe:
    c_1 = df1_name + '.' + series_1.name + '_values' 
    c_2 = df2_name + '.' + series_2.name
    c_2 = 'matches_in_'+c_2+' '+mode+' '+str(n_matches)
    
    # Turning input series to lists:
    l_1 = list(series_1)
    l_2 = list(series_2)
    
    # Calculating mathces in series_2 for each element in series_1:
    matches = [l_2.count(i) for i in l_1]
    
    # Create DataFrame for series_1 vs matches in series_2
    df = pd.DataFrame({c_1: l_1, c_2: matches})
    
    df = df.drop_duplicates()

    if mode == '=':
        return df[df[c_2] == n_matches]
    if mode == '>':
        return df[df[c_2] > n_matches]
    if mode == '<':
        return df[df[c_2] < n_matches]
    if mode == '!=':
        return df[df[c_2] != n_matches]

In [10]:
# Try it out here:

# df = pd.DataFrame(
#     {
#         'a': [1, 2, 3, 4],
#         'b': [1, 1, 3, 5]
#     }
# )

# series_1 = pd.Series({'a': [1, 2, 3, 4]})
# series_2 = pd.Series({'b': [1, 1, 3, 5, 7]})
# series_3 = pd.Series([1, 2, 3, 4])
# series_4 = pd.Series([1, 1, 3, 5, 7])
# foreign_k2k(df.a, df.b)

# 4. Need Trimming?
A long column of strings sometimes starts with undesirable characters like brackets or others. Let's quickly check what are the initials of our strings:

In [11]:
def initials(s):
    # Given a Series of strings, it retruns all unique first characters
    s = s.map(lambda x: x[0] if isinstance(x, str) else x)
    return s.value_counts(dropna = False)

In [13]:
def finals(s):
    # Given a Series of strings, it retruns all unique first characters
    s = s.map(lambda x: x[-1] if isinstance(x, str) else x)
    return s.value_counts(dropna = False)